In [96]:
import numpy as np
import random
from copy import copy, deepcopy

In [ ]:
nb_gen = 100
nb_pop = 10
mutation_rate = 0.2

In [118]:
def init_grid():
    map = np.zeros((4,4))
    add_bloc(map)
    return map

In [119]:
def add_bloc(map):
    nb = np.count_nonzero(map == 0)
    temp = random.random()
    if nb != 0 :
        rdx = random.randint(0,3)
        rdy = random.randint(0,3)
        while map[rdx][rdy] != 0:
            rdx = random.randint(0,3)
            rdy = random.randint(0,3)
        if temp >0.70 :
            map[rdx][rdy] = 4
        else :
            map[rdx][rdy] = 2

In [120]:
def is_over(map):
    if np.count_nonzero(map == 0):
        return 0
    for i in range(0,3):
        for j in range(0,3):
            if map[i][j] == map[i][j+1] or map[i][j] == map[i+1][j] :
                return 0
    return 1

In [121]:
def move_to(key, map):
    score_added = 0
    if key == 0:
        #print("moving up")
        map = map.transpose()
        map = np.fliplr(map)
        score_added = move_to(3,map)
        map = map.transpose()
        map = np.fliplr(map)
    elif key == 1:
        #print("moving down")
        map = map.transpose()
        score_added = move_to(3,map)
        map = map.transpose()
    elif key == 2:
        #print("moving left")
        map = np.fliplr(map)
        score_added = move_to(3,map)
        map = np.fliplr(map)
    elif key == 3:
        #print("moving right")
        for i in range(0,4):
            nb =np.count_nonzero(map[i] == 0)
            a = np.zeros(nb)
            b = np.zeros(4-nb)
            k = 0
            for element in map[i]:
                if element !=0:
                    b[k] = element
                    k+=1
            map[i] = np.concatenate((a,b))
            for j in range(0,3):
                if not(map[i][j] == map[i][j+1] and map[i][j+1] !=0):
                    continue
                else :
                    #print("là ça ajoute")
                    map[i][j+1] = 2*map[i][j+1]
                    map[i][0:j+1] = np.roll(map[i][0:j+1],1)
                    map[i][0] = 0
                    score_added += 2*map[i][j+1]
                    #print(i,j+1)
    return score_added

In [136]:
class Bot : 
    def __init__(self):
        self.A = np.random.random((10,16))
        self.B = np.random.random((10,10))
        self.C= np.random.random((10,4))
        self.last_fitness = 0

    def make_decision(self,grid):
        map_vect = np.reshape(grid,16)
        output = self.A.dot(map_vect).dot(self.B).dot(self.C)
        i = np.argmax(output)
        grid_copy = deepcopy(grid)
        move_to(i , grid_copy)
        #print(output)
        if (grid_copy == grid).all():
            if np.array_equal(grid_copy, copy):
                #print("useless move")
                i = random.randint(0,3)
        return i 


    def run_neural_network(self):
        score = 0
        nb_coups = 0
        grid = init_grid()
        #print(grid)
        for i in range(100):
            nb_coups = 0
            grid = init_grid()
            while (not is_over(grid) and nb_coups <= 100 ):
                key = self.make_decision(grid)
                #print("key selected :                "+str(key))
                score += move_to(key,grid)
                add_bloc(grid)
                #print(grid)
                nb_coups += 1
                #print("score : "+str(score))
                #print("next")
        return score
    
    def evaluate(self):
        self.fitness = self.run_neural_network()
        
    def mutate(self):
        child = Bot()
        child.A = copy(self.A)
        child.B = copy(self.B)
        child.C = copy(self.C)
        for i in child.A:
            if random.random() < mutation_rate:
                i = random.random()
        for i in child.B:
            if random.random() < mutation_rate:
                i = random.random()
        for i in child.C:
            if random.random() < mutation_rate:
                i = random.random()
    
        return child


            


In [137]:
def one_plus_one(nb_gen,mutation_rate):
    fits = np.zeros(nb_gen)
    parent = Bot()
    parent.evaluate()
    
    for i in range(nb_gen):
        print("generation "+str(i))
        child = parent.mutate()
        child.evaluate()
        print("fitness :"+str(child.fitness))

        if child.fitness >= parent.fitness:
            print("il y a selection")
            parent = child
        fits[i] = parent.fitness
    print(fits)

In [139]:
one_plus_one(nb_gen, mutation_rate)

generation 0
fitness :26928.0
il y a selection
generation 1
fitness :26456.0
generation 2
fitness :25440.0
generation 3
fitness :22896.0
generation 4
fitness :25312.0
generation 5
fitness :25752.0
generation 6
fitness :26184.0
generation 7
fitness :25224.0
generation 8
fitness :26720.0
generation 9
fitness :25696.0
generation 10
fitness :24400.0
generation 11
fitness :25368.0
generation 12
fitness :26216.0
generation 13
fitness :25280.0
generation 14
fitness :26064.0
generation 15
fitness :25928.0
generation 16
fitness :24152.0
generation 17
fitness :25944.0
generation 18
fitness :26424.0
generation 19
fitness :27432.0
il y a selection
generation 20
fitness :24608.0
generation 21
fitness :26520.0
generation 22
fitness :26512.0
generation 23
fitness :25776.0
generation 24
fitness :26704.0
generation 25
fitness :24416.0
generation 26
fitness :23840.0
generation 27
fitness :24008.0
generation 28
fitness :23624.0
generation 29
fitness :22432.0
generation 30
fitness :24416.0
generation 31
f